In [13]:
print("Intelligent Home Energy Management System")

#Required Libraries
import sys
import numpy as np
import skfuzzy as fuzz
from skfuzzy import control as ctrl
import requests
import datetime
import matplotlib.pyplot as plt

#API Keys
OPENWEATHER_API_KEY = "7c730aa2f65b08b884f8dfa7f125bfb8" 

# Dynamically get today's date in UTC
today = datetime.datetime.utcnow().strftime("%Y-%m-%d")

# Octopus Agile Tariff URL for today's price
OCTOPUS_API_URL = (
    f"https://api.octopus.energy/v1/products/AGILE-18-02-21/"
    f"electricity-tariffs/E-1R-AGILE-18-02-21-C/standard-unit-rates/"
    f"?period_from={today}T00:00Z&period_to={today}T23:59Z"
)


#Location Detection
def get_location_info():
    try:
        response = requests.get("http://ip-api.com/json/")
        data = response.json()
        city = data['city']
        lat = data['lat']
        lon = data['lon']
        print(f"Detected Location: {city} ({lat}, {lon})")
        return city, lat, lon
    except Exception as e:
        print("Failed to get location info:", e)
        return "London", 51.5074, -0.1278  # fallback to London

#Weather Data
def get_weather(lat, lon):
    url = f"http://api.openweathermap.org/data/2.5/weather?lat={lat}&lon={lon}&appid={OPENWEATHER_API_KEY}&units=metric"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        temperature = data['main']['temp']
        humidity = data['main']['humidity']
        return temperature, humidity
    else:
        print("Weather API Error:", response.status_code)
        return None, None

#Energy Pricing
def get_energy_price():
    response = requests.get(OCTOPUS_API_URL)
    if response.status_code == 200:
        data = response.json()
        if data['results']:
            return data['results'][0]['value_exc_vat'] / 100  # Convert pence to pounds
        else:
            print("No tariff data found.")
            return None
    else:
        print("Energy API Error:", response.status_code)
        return None

#Example Use 
city, lat, lon = get_location_info()
current_temp, current_humidity = get_weather(lat, lon)
current_price = get_energy_price()
current_hour = datetime.datetime.now().hour

#Display Results
print(f"Temperature in {city}: {current_temp}°C")
print(f"Humidity in {city}: {current_humidity}%")
print(f"Energy Price: £{current_price:.4f} per kWh")
print(f"Current Hour: {current_hour}")


Intelligent Home Energy Management System
Detected Location: Southwark (51.4971, -0.0482)
Temperature in Southwark: 19.63°C
Humidity in Southwark: 58%
Energy Price: £0.1648 per kWh
Current Hour: 12
